In [1]:
import pandas as pd

def CalculateSessionMGPValuesOverTime(results, minScore, maxScore):
    '''
    Calcula el MGP de un grupo de resultados a partir 
    del indice de rendimiento, y devuelve una lista con
    los valores correspondientes al MGP de la sesion.
    
    Parameters:
    results (pandas.DataFrame): Una Series con los 
    indices de rendimiento de los cuales se quiere 
    obtener el MGP, ordenados de acuerdo al orden en que
    fueron jugados.
    minScore (int): El indice de rendimiento minimo que 
    existe en esta dificultad.
    maxScore (int): El indice de rendimiento maximo que 
    existe en esta dificultad.
    
    Returns:
    float[]: Listado con los MGP de la sesion.
    '''    
    
    scaledResults = pd.concat([pd.Series([minScore, maxScore]), results])
    mgpValues = []
    
    for i in range(2, scaledResults.size):
        currentPI = scaledResults.iloc[i]
        previousPIs = scaledResults.iloc[:i]
        mgpValues.append(CalculateSessionMGP(currentPI, previousPIs))
            
    return mgpValues

def CalculateSessionMGP(currentScore, previousScores):
    '''
    Calcula el MGP de un resultado a partir del indice de
    rendimiento y los resultados previos.
    
    Note: Para que funcione adecuadamente, la lista de
    scores previos tiene que contener los valores frontera
    (minimo y maximo) de la dificultad. De otro modo, el
    MGP resultante estara sesgado.
    
    Parameters:
    currentScore (int): El indice de rendimiento para el 
    cual se quiere obtener el MGP.
    previousScores (pandas.Series): Series con los
    indices de rendimiento previos del paciente en esta
    dificultad, ordenados segun fueron jugados.
    
    Returns:
    float: MGP de la sesion.
    '''
    df = pd.concat([previousScores, pd.Series([currentScore])])
    if df.min() == df.max():
        return 0.5
    
    normalizedDf = (df-df.min())/(df.max()-df.min())
    
    return normalizedDf.iloc[-1]

def GetDifficultyMinAndMaxValues(difficulty, column="Puntaje"):
    df = GetAllResultsFromDifficulties([difficulty])[column]
    return df.min(), df.max()

def RemoveNaNValues(dataframe, column="Puntaje"):
    mask = ~dataframe[column].isna()
    return dataframe[mask]